In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#A cleaned version Brazil MST data (only for 2005)
data = pd.read_stata('C:/Users/user/Desktop/Advanced Data Analysis with Python 9.9.22/Datasets/Brazil MST data 2005.dta')
data.columns

In [ ]:
data

In [ ]:
import seaborn as sns
sns.scatterplot(x="invasions_count_cum", y="BF_coverage1000", data=data)

In [ ]:
sns.scatterplot(x="invasions_count", y="BF_coverage1000", data=data)

In [ ]:
sns.lmplot(x="invasions_count", y="BF_coverage1000", data=data)

In [ ]:
sns.displot(x='BF_coverage1000', row='invasions_dum', data=data, linewidth=0, kde=True)

In [ ]:
sns.kdeplot(x='BF_coverage1000', hue='invasions_dum', data=data, shade=True);

In [ ]:
#Frequency
data.groupby('invasions_count')['invasions_count'].count()

### IV2SLS model

In [ ]:
#pip install statsmodels

In [ ]:
data = data.dropna(axis=0)

In [ ]:
import statsmodels.api as sm

y = data['BF_coverage1000']
X = data['log_invasions_count_cum']
Z = data['log_intended_lands1995']

#### What if we run OLS?

In [ ]:
OLS =  sm.OLS(y, X).fit()
OLS.summary()

#### IV2SLS by hand

In [ ]:
first_stage = sm.OLS(X, Z).fit()
predicted_X = first_stage.predict(Z)

second_stage = sm.OLS(y, predicted_X).fit()
second_stage.summary()

In [ ]:
first_stage.summary()

In [ ]:
#pip install linearmodels

#### Use already existing libraries

In [ ]:
from statsmodels.sandbox.regression.gmm import IV2SLS

In [ ]:
iv2sls = IV2SLS(data['BF_coverage1000'], data['log_invasions_count_cum'], data['log_intended_lands1995']).fit()

In [ ]:
iv2sls.params

In [ ]:
import linearmodels

In [ ]:
from linearmodels.iv import IV2SLS

In [ ]:
formula = 'BF_coverage1000 ~ [log_invasions_count_cum ~ log_intended_lands1995]'
iv2sls = IV2SLS.from_formula(formula, data).fit()

In [ ]:
def parse(model, exog="log_invasions_count_cum"):
    param = model.params[exog]
    se = model.std_errors[exog]
    p_val = model.pvalues[exog]
    print(f"Parameter: {param}")
    print(f"SE: {se}")
    print(f"95 CI: {(-1.96*se,1.96*se) + param}")
    print(f"P-value: {p_val}")
    
parse(iv2sls)

In [ ]:
iv2sls

In [ ]:
formula = 'BF_coverage1000 ~ 1 + log_income + ln_agriculturalproductivity + PT_voteshare + infantmortality + [log_invasions_count_cum ~ log_intended_lands1995]'
iv2sls = IV2SLS.from_formula(formula, data).fit()

In [ ]:
iv2sls